In [5]:
!pip install pycocotools

  Using cached pycocotools-2.0.8-cp312-cp312-macosx_10_9_universal2.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 1.9 MB/s eta 0:00:00a 0:00:01


In [18]:
import os
import numpy as np
from pycocotools.coco import COCO
from PIL import Image

def preprocess_image(image_path, target_size=(224, 224)):
    #Load an image, resize it to the target size, and normalize pixel values.
    image = Image.open(image_path).convert('RGB')
    image = image.resize(target_size)
    image_array = np.array(image) / 255.0  # Normalize to [0,1]
    return image_array

def load_coco_annotations(coco_json_path, images_dir, target_size=(224, 224)):
    #Load images and annotations from a COCO JSON file, preprocess images, and return data ready for training.
    coco = COCO(coco_json_path)
    images = []
    annotations = []
    
    # Loop over all images in the COCO dataset
    for img_id in coco.getImgIds():
        img_info = coco.loadImgs(img_id)[0]
        image_path = os.path.join(images_dir, img_info['file_name'])
        
        # Preprocess the image
        processed_image = preprocess_image(image_path, target_size)
        images.append(processed_image)
        
        # Get annotations for the current image
        ann_ids = coco.getAnnIds(imgIds=img_id)
        img_annotations = []
        
        for ann in coco.loadAnns(ann_ids):
            if 'bbox' in ann:
                # Process bounding box annotations (x, y, width, height)
                x, y, w, h = ann['bbox']
                x_scale = target_size[0] / img_info['width']
                y_scale = target_size[1] / img_info['height']
                
                # Scale bounding boxes to the new image size
                scaled_bbox = [x * x_scale, y * y_scale, w * x_scale, h * y_scale]
                img_annotations.append(scaled_bbox)
        
        annotations.append(img_annotations)
    
    return np.array(images), annotations

# Example usage
coco_json_path = '/Users/shreejith/suryaprojects/Dental_ML_Diagnosis_Project/train_coco.json'
images_dir = 'path/to/images'
target_size = (224, 224)

images, annotations = load_coco_annotations(coco_json_path, images_dir, target_size)

print(f"Processed {len(images)} images and {len(annotations)} sets of annotations.")

loading annotations into memory...


NotADirectoryError: [Errno 20] Not a directory: '/Users/shreejith/suryaprojects/Dental_ML_Diagnosis_Project/train_coco.json/annotations/instances_train_coco.json'